In [0]:
!pip install  kaggle
import os, json
from google.colab import files
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score 
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import cross_val_score

In [0]:


uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
!mkdir ~/.kaggle/  



Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [0]:
!ls
!mv kaggle.json ~/.kaggle/kaggle.json

kaggle.json  sample_data


In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [0]:


with open('/root/.kaggle/kaggle.json') as json_file:  
    d = json.load(json_file)
    os.environ['KAGGLE_USERNAME'] = d["username"]
    os.environ['KAGGLE_KEY'] = d["key"]

!export | grep KAGGLE

declare -x KAGGLE_KEY="f2fddbaeb9a502580966c4d13630f7d1"
declare -x KAGGLE_USERNAME="itobetter"


In [0]:
!kaggle competitions list

ref                                                deadline             category            reward  teamCount  userHasEntered  
-------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                                   2030-01-01 00:00:00  Getting Started  Knowledge       3043           False  
titanic                                            2030-01-01 00:00:00  Getting Started  Knowledge      11385            True  
house-prices-advanced-regression-techniques        2030-01-01 00:00:00  Getting Started  Knowledge       4569           False  
imagenet-object-localization-challenge             2029-12-31 07:00:00  Research         Knowledge         37           False  
competitive-data-science-predict-future-sales      2019-12-31 23:59:00  Playground           Kudos       3049           False  
two-sigma-financial-news                           2019-07-15 23:59:00  Featured          $100,000      

In [0]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 52.1MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.0MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.87MB/s]


In [0]:

df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:

# determinate thats the cabin is more null values than 60 porcent
df.describe()
df.Cabin.describe()
df.Cabin.isnull().mean()
# delete rows Embarked null
df.dropna(subset=['Embarked'], inplace=True)
# separate data for kand of age
df['Age'].fillna(df['Age'].mean(), inplace = True)
age_bins = [0,14,30,50,80]
age_labels=['0-14','15-30','30-80','80-110']
pd_eva = pd.concat([
    df, pd.get_dummies(df["Sex"]), 
    pd.get_dummies(df['Pclass'], prefix='Pclass'),
    pd.get_dummies(df['Embarked'], prefix='Embarked'),
    pd.get_dummies(pd.cut(df['Age'], bins=age_bins, labels=age_labels, include_lowest=True))
  ], axis=1
)
pd_eva.drop(
    ['Name','Sex',
     'Age','Cabin',
     'PassengerId','Ticket', 
     'Embarked'
    ], axis=1, inplace=True
)
#pd_eva["Fare"] = np.log(pd_eva["Fare"])
pd_eva.reset_index(drop=True, inplace=True)
no_survived = pd_eva.Survived.value_counts()[1]
non_survived = pd_eva[pd_eva['Survived'] == 0]
survived = pd_eva[pd_eva['Survived'] == 1]
non_survived_selected =  non_survived.sample(no_survived)
last_df = pd.concat([non_survived_selected, survived])

last_df.Survived.unique()

array([0, 1])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(last_df.drop(["Survived"], axis=1),last_df["Survived"], test_size=0.2)
pd.concat([X_train, y_train],axis=1).head()

,Pclass,SibSp,Parch,Fare,female,male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,0-14,15-30,30-80,80-110,Survived
693,1,0,0,26.5500,0,1,1,0,0,0,0,1,0,0,0,1,0
279,3,0,0,7.7500,0,1,0,0,1,0,1,0,0,0,0,1,0
369,1,1,0,55.4417,0,1,1,0,0,1,0,0,0,1,0,0,1
452,1,1,0,89.1042,0,1,1,0,0,1,0,0,0,0,1,0,1
317,1,0,2,164.8667,1,0,1,0,0,0,0,1,0,0,1,0,1


In [0]:

m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
cross_val_score(m,X_train, y_train)
m.feature_importances_
m.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0])